### For data where there is atleast 1 click, view important features for Impressions

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Excel file to explore its content and structure
campaign_data = pd.read_csv('Amazon Ads All.csv')

# Display the first few rows of the dataframe and the column names to understand the dataset
campaign_data.head(), campaign_data.columns


(   ADV_ID    AD_ID         SKU     placement_slot  impressions  clicks  \
 0   84373  6370624  9985955750  offsite_mobileweb         1089       0   
 1   84373  1295011  7875390725    offsite_desktop         2043       0   
 2   84373  6370624  9985955750  offsite_mobileweb         1452       0   
 3   84373  1295011  7875390725    offsite_desktop         1312       0   
 4   84373  1295011  7875390725  offsite_mobileapp            8       0   
 
    auction_cost  adjusted_cost hit_day_utc  \
 0      10.89000        8.79000    02/05/24   
 1      26.61464       19.44791    01/05/24   
 2      14.52000       11.68400    01/05/24   
 3      16.59940       11.94286    30/04/24   
 4       0.11608        0.07161    01/05/24   
 
                                  targeting_secondary    vertical  \
 0       category=Automotive Replacement Control Arms  Automotive   
 1         category=Automotive Replacement Brake Kits  Automotive   
 2       category=Automotive Replacement Control Arms  Au

### CLEANING

In [2]:
## CLEANING
# Attempt to parse dates in the '%d/%m/%y' format
campaign_data['hit_day_utc_parsed'] = pd.to_datetime(campaign_data['hit_day_utc'], format='%d/%m/%y', errors='coerce')

# Parse remaining unparsed dates in the '%Y-%m-%d %H:%M:%S' format
mask = campaign_data['hit_day_utc_parsed'].isna()
campaign_data.loc[mask, 'hit_day_utc_parsed'] = pd.to_datetime(campaign_data.loc[mask, 'hit_day_utc'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Check for any remaining non-parsed dates
remaining_non_parsed = campaign_data[campaign_data['hit_day_utc_parsed'].isna()]
if not remaining_non_parsed.empty:
    print("Remaining non-parsed dates:")
    print(remaining_non_parsed)

# Print unique months in the parsed 'hit_day_utc_parsed' column
print("Unique months in 'hit_day_utc_parsed':")
print(campaign_data['hit_day_utc_parsed'].dt.month.unique())

# Extract month and day of the week from parsed dates
campaign_data['month'] = campaign_data['hit_day_utc_parsed'].dt.month
campaign_data['day_of_week'] = campaign_data['hit_day_utc_parsed'].dt.dayofweek

campaign_data['month_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%B')  # Get month names
campaign_data['day_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%A')    # Get day names

# Fill missing values in 'vertical' and 'sub_vertical' with 'Unknown'
campaign_data['vertical'].fillna('Unknown', inplace=True)
campaign_data['sub_vertical'].fillna('Unknown', inplace=True)

# Identify the last 6 columns
last_6_columns = campaign_data.columns[-6:]

# Fill missing values in the last 6 columns with 0
campaign_data[last_6_columns] = campaign_data[last_6_columns].fillna(0)

# Display the cleaned dataset
print("Cleaned Campaign Data Sample:")
print(campaign_data.head())

# Inspect the 'hit_day_utc' column to understand its format
print("Sample of 'hit_day_utc' column:")
print(campaign_data['hit_day_utc'].head(10))

# Filter the data where clicks is at least 1
campaign_data = campaign_data[campaign_data['clicks'] >= 1]


Unique months in 'hit_day_utc_parsed':
[5 4 3 2]


/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_48897/2588757257.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  campaign_data['vertical'].fillna('Unknown', inplace=True)
/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_48897/2588757257.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

Cleaned Campaign Data Sample:
   ADV_ID    AD_ID         SKU     placement_slot  impressions  clicks  \
0   84373  6370624  9985955750  offsite_mobileweb         1089       0   
1   84373  1295011  7875390725    offsite_desktop         2043       0   
2   84373  6370624  9985955750  offsite_mobileweb         1452       0   
3   84373  1295011  7875390725    offsite_desktop         1312       0   
4   84373  1295011  7875390725  offsite_mobileapp            8       0   

   auction_cost  adjusted_cost hit_day_utc  \
0      10.89000        8.79000    02/05/24   
1      26.61464       19.44791    01/05/24   
2      14.52000       11.68400    01/05/24   
3      16.59940       11.94286    30/04/24   
4       0.11608        0.07161    01/05/24   

                                 targeting_secondary  ...  \
0       category=Automotive Replacement Control Arms  ...   
1         category=Automotive Replacement Brake Kits  ...   
2       category=Automotive Replacement Control Arms  ...   
3  c

### Predicting impressions  considering clicks in features

In [3]:
# Prepare the dataset

# Display columns to ensure the correct ones are present
print("Columns in Campaign Data:")
print(campaign_data.columns)

# Check if 'placement_slot', 'targeting_secondary', 'vertical', and 'sub_vertical' exist in the columns before encoding
categorical_columns = ['placement_slot', 'targeting_secondary', 'vertical', 'sub_vertical', 'month_name', 'day_name']
existing_categorical_columns = [col for col in categorical_columns if col in campaign_data.columns]

if existing_categorical_columns:
    campaign_data = pd.get_dummies(campaign_data, columns=existing_categorical_columns, drop_first=True)

# Select features and target variable
features = ['clicks'] + \
           [col for col in campaign_data.columns if col.startswith('placement_slot_')] + \
           [col for col in campaign_data.columns if col.startswith('targeting_secondary_')] + \
           [col for col in campaign_data.columns if col.startswith('vertical_')] + \
           [col for col in campaign_data.columns if col.startswith('sub_vertical_')] + \
           [col for col in campaign_data.columns if col.startswith('month_name_')] + \
           [col for col in campaign_data.columns if col.startswith('day_name_')]

target = 'impressions'

X = campaign_data[features]
y = campaign_data[target]

# Handle missing values using SimpleImputer
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

if len(numeric_features) > 0:
    numeric_imputer = SimpleImputer(strategy='mean')
    X[numeric_features] = numeric_imputer.fit_transform(X[numeric_features])

if len(categorical_features) > 0:
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    X[categorical_features] = categorical_imputer.fit_transform(X[categorical_features])

# Check if any missing values remain
print("Missing values in X after imputation:")
print(X.isna().sum().sum())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Gradient Boosting Regressor
gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(X_train, y_train)

# Predict on the test set
y_pred = gbr.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display the evaluation metrics
evaluation_metrics = {
    'Mean Absolute Error': mae,
    'Mean Squared Error': mse,
    'R-squared': r2
}

print("Evaluation Metrics:")
print(pd.DataFrame([evaluation_metrics]))

# Display feature importances
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': gbr.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance for Gradient Boosting Regressor:")
pd.set_option('display.max_rows', None)  # Ensure all rows are displayed
print(feature_importance_df)

Columns in Campaign Data:
Index(['ADV_ID', 'AD_ID', 'SKU', 'placement_slot', 'impressions', 'clicks',
       'auction_cost', 'adjusted_cost', 'hit_day_utc', 'targeting_secondary',
       'vertical', 'sub_vertical', 'ntb_click_attributed_units_sold',
       'ntb_view_attributed_units_sold', 'ntb_click_attributed_orders',
       'ntb_view_attributed_orders', 'view_attributed_units_sold',
       'view_attributed_orders', 'hit_day_utc_parsed', 'month', 'day_of_week',
       'month_name', 'day_name'],
      dtype='object')


/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_46800/4249669772.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_features] = numeric_imputer.fit_transform(X[numeric_features])


Missing values in X after imputation:
0
Evaluation Metrics:
   Mean Absolute Error  Mean Squared Error  R-squared
0          1791.863392        6.338277e+06   0.708437

Feature Importance for Gradient Boosting Regressor:
                                                Feature    Importance
0                                                clicks  2.810726e-01
2                      placement_slot_offsite_mobileweb  1.774951e-01
980   targeting_secondary_category=Women's Running S...  1.016594e-01
517      targeting_secondary_category=Kids' Electronics  6.115476e-02
1                      placement_slot_offsite_mobileapp  6.053933e-02
788   targeting_secondary_category=Sheet & Pillowcas...  6.022911e-02
1042                    sub_vertical_Grocery - Beverage  2.873284e-02
365      targeting_secondary_category=Foundation Makeup  2.458559e-02
249   targeting_secondary_category=Desk Accessories ...  2.220376e-02
713   targeting_secondary_category=Pipe Fittings & P...  1.744551e-02
425   tar

### Predicting impressions without considering clicks in features, no verticals

MSE is greater than before

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np


ntb_data = campaign_data[campaign_data['clicks'] >= 1]
# Define features and target for NTB reach
features_ntb = ['targeting_secondary', 
                'placement_slot', 'month_name', 'day_name']
target_ntb = 'impressions'

# Prepare the data
X_ntb = ntb_data[features_ntb]
y_ntb = ntb_data[target_ntb]

# One-hot encode categorical features
preprocessor_ntb = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['targeting_secondary', 'placement_slot', 'month_name', 'day_name'])
    ],
    remainder='passthrough'
)

# Define and train the Gradient Boosting Regressor for NTB reach
model_ntb = Pipeline(steps=[
    ('preprocessor', preprocessor_ntb),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

X_train_ntb, X_test_ntb, y_train_ntb, y_test_ntb = train_test_split(X_ntb, y_ntb, test_size=0.2, random_state=42)
model_ntb.fit(X_train_ntb, y_train_ntb)

# Predict and evaluate the NTB reach model
y_pred_ntb = model_ntb.predict(X_test_ntb)
mse_ntb = mean_squared_error(y_test_ntb, y_pred_ntb)
mae_ntb = mean_absolute_error(y_test_ntb, y_pred_ntb)
r2_ntb = r2_score(y_test_ntb, y_pred_ntb)

# Display the evaluation metrics for NTB reach model
print(f"NTB Reach Model - Mean Squared Error: {mse_ntb}")
print(f"NTB Reach Model - Mean Absolute Error: {mae_ntb}")
print(f"NTB Reach Model - R-squared: {r2_ntb}")

# Get feature names after one-hot encoding
all_feature_names = model_ntb.named_steps['preprocessor'].transformers_[0][1].get_feature_names_out(['targeting_secondary', 'placement_slot', 'month_name', 'day_name'])
# all_feature_names = np.concatenate((encoded_feature_names, [ 'adjusted_cost']))

# Calculate feature importance
importance_ntb = model_ntb.named_steps['regressor'].feature_importances_
feature_importance_ntb = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': importance_ntb
}).sort_values(by='Importance', ascending=False)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
print("Feature Importance for NTB Reach Model:")
print(feature_importance_ntb)

NTB Reach Model - Mean Squared Error: 8098505.422630209
NTB Reach Model - Mean Absolute Error: 2060.191435422612
NTB Reach Model - R-squared: 0.6274662922373192
Feature Importance for NTB Reach Model:
                                                                            Feature  \
751                            targeting_secondary_category=Sheet & Pillowcase Sets   
932                              targeting_secondary_category=Women's Running Shoes   
765                targeting_secondary_category=Single-Serve Coffee Capsules & Pods   
491                                  targeting_secondary_category=Kids' Electronics   
978                                                  placement_slot_offsite_desktop   
715                  targeting_secondary_category=Probiotic Nutritional Supplements   
377                                     targeting_secondary_category=Golf Equipment   
402                              targeting_secondary_category=Hair Styling Products   
988             

### Impressions with no verticals considering clicks in features

In [3]:
# Prepare the dataset

# Display columns to ensure the correct ones are present
print("Columns in Campaign Data:")
print(campaign_data.columns)

# Check if 'placement_slot', 'targeting_secondary', 'vertical', and 'sub_vertical' exist in the columns before encoding
categorical_columns = ['placement_slot', 'targeting_secondary', 'month_name', 'day_name']
existing_categorical_columns = [col for col in categorical_columns if col in campaign_data.columns]

if existing_categorical_columns:
    campaign_data = pd.get_dummies(campaign_data, columns=existing_categorical_columns, drop_first=True)

# Select features and target variable
features = ['clicks'] + \
           [col for col in campaign_data.columns if col.startswith('placement_slot_')] + \
           [col for col in campaign_data.columns if col.startswith('targeting_secondary_')] + \
           [col for col in campaign_data.columns if col.startswith('month_name_')] + \
           [col for col in campaign_data.columns if col.startswith('day_name_')]

target = 'impressions'

X = campaign_data[features]
y = campaign_data[target]

# Handle missing values using SimpleImputer
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

if len(numeric_features) > 0:
    numeric_imputer = SimpleImputer(strategy='mean')
    X[numeric_features] = numeric_imputer.fit_transform(X[numeric_features])

if len(categorical_features) > 0:
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    X[categorical_features] = categorical_imputer.fit_transform(X[categorical_features])

# Check if any missing values remain
print("Missing values in X after imputation:")
print(X.isna().sum().sum())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Gradient Boosting Regressor
gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(X_train, y_train)

# Predict on the test set
y_pred = gbr.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display the evaluation metrics
evaluation_metrics = {
    'Mean Absolute Error': mae,
    'Mean Squared Error': mse,
    'R-squared': r2
}

print("Evaluation Metrics:")
print(pd.DataFrame([evaluation_metrics]))

# Display feature importances
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': gbr.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance for Gradient Boosting Regressor:")
pd.set_option('display.max_rows', None)  # Ensure all rows are displayed
print(feature_importance_df)

Columns in Campaign Data:
Index(['ADV_ID', 'AD_ID', 'SKU', 'placement_slot', 'impressions', 'clicks',
       'auction_cost', 'adjusted_cost', 'hit_day_utc', 'targeting_secondary',
       'vertical', 'sub_vertical', 'ntb_click_attributed_units_sold',
       'ntb_view_attributed_units_sold', 'ntb_click_attributed_orders',
       'ntb_view_attributed_orders', 'view_attributed_units_sold',
       'view_attributed_orders', 'hit_day_utc_parsed', 'month', 'day_of_week',
       'month_name', 'day_name'],
      dtype='object')


/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_48897/3698460439.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_features] = numeric_imputer.fit_transform(X[numeric_features])


Missing values in X after imputation:
0
Evaluation Metrics:
   Mean Absolute Error  Mean Squared Error  R-squared
0           1817.52661        6.508666e+06   0.700599

Feature Importance for Gradient Boosting Regressor:
                                                Feature    Importance
0                                                clicks  2.844079e-01
2                      placement_slot_offsite_mobileweb  1.817791e-01
980   targeting_secondary_category=Women's Running S...  1.009518e-01
788   targeting_secondary_category=Sheet & Pillowcas...  7.027134e-02
517      targeting_secondary_category=Kids' Electronics  6.248335e-02
1                      placement_slot_offsite_mobileapp  6.157331e-02
802   targeting_secondary_category=Single-Serve Coff...  4.011623e-02
365      targeting_secondary_category=Foundation Makeup  2.570226e-02
249   targeting_secondary_category=Desk Accessories ...  2.246029e-02
713   targeting_secondary_category=Pipe Fittings & P...  1.850838e-02
751   tar